In [2]:
import pandas as pd
from darts import TimeSeries
from pathlib import Path

In [12]:
cwd=Path.cwd()
data= pd.read_csv(cwd.parent.parent / 'Data' /'Train'/'train1990s.csv',parse_dates=[0],date_format='%m%Y')
# display(data)


series = TimeSeries.from_dataframe(data,time_col='observation_date')
display(series.n_timesteps)
# transformedCols=[]
# for i in data.columns:
#     match=re.findall(r'fred_.*_.*',i)
#     if match!=[]:
#         transformedCols.append(match[0])

# unmodifiedDf= data.drop(transformedCols,axis=1)
# split= round(unmodifiedDf.shape[0]*0.9)
# train= unmodifiedDf.iloc[:split+1,:]
# valid= unmodifiedDf.iloc[split+1:,:]

372